In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import json
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re

!pip install flair
from flair.models import TextClassifier
from flair.data import Sentence


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 71.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
columns = ['business_id', 'text_top10', 'bart_summary', 'beam_size', 'no_repeat_ngrams']
gridsearch = pd.DataFrame(columns = columns)

beam_sizes = [1,3,5]
no_repeat_ngram_sizes = [3,5,7]

for beam_size in beam_sizes:
  for no_repeat_ngrams in no_repeat_ngram_sizes:
    df = pd.read_csv('/content/drive/MyDrive//W266 Notes/W266 Final Project/BART_GridSearch/bart_summary_sample_b{}_n{}.csv'.format(beam_size, no_repeat_ngrams))
    df['beam_size'] = beam_size
    df['no_repeat_ngrams'] = no_repeat_ngrams

    gridsearch = pd.concat([gridsearch, df], axis = 0)


gridsearch.reset_index(drop = True, inplace = True)

gridsearch.head()

,business_id,text_top10,bart_summary,beam_size,no_repeat_ngrams
0,hHzKMfJe7uuw_7WIEHEwBQ,Well I haven't been to PJs oyster bar since th...,PJ's oyster bar has moved to the same location...,1,3
1,49dVbKxmhJ08KGH--9getA,I kind of mulled over this review for the bett...,"Ramen Sho is a hidden gem in Boise, Idaho. The...",1,3
2,WNqqyUtEmrqCBEHNV9sj5Q,First time visit to Bordentown and Marcellos! ...,Marcello's is a great Italian restaurant in Bo...,1,3
3,V66EXhrxKanesMhoOqsORw,This was great! It's a wonderful find in a pre...,Shorty's Sunflower Cafe is a great find in a d...,1,3
4,YVvm1Q3ir5TET0f66FOX_g,Hands down the most authentic Italian restaura...,"La Terrazza is a small, intimate Italian resta...",1,3


In [3]:
def flair_sentiment(input_string):
  '''Uses English sentiment classifier from Flair NLP library to calculate sentiment % for a string'''
  classifier = TextClassifier.load('en-sentiment')
  sentence = Sentence(input_string)
  classifier.predict(sentence)

  sentiment = sentence.labels[0].value
  score = round(sentence.labels[0].score,3)
  return sentiment, score

In [4]:
gridsearch[['BART_sentiment', 'BART_score']] = gridsearch['bart_summary'].apply(flair_sentiment).apply(pd.Series)

gridsearch.to_csv('/content/drive/MyDrive/W266 Notes/W266 Final Project/BART_gridsearch_2.csv', index = False)

2023-08-02 22:37:32,896 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-distilbert/sentiment-en-mix-distillbert_4.pt not found in cache, downloading to /tmp/tmpsx57lw02


100%|██████████| 253M/253M [00:21<00:00, 12.5MB/s]

2023-08-02 22:37:55,334 copying /tmp/tmpsx57lw02 to cache at /root/.flair/models/sentiment-en-mix-distillbert_4.pt


2023-08-02 22:37:56,112 removing temp file /tmp/tmpsx57lw02
